<a href="https://colab.research.google.com/github/mohan-mj/Autoencoder/blob/master/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/mohan-mj/Autoencoder/blob/master/2.png?raw=true)

1- Encoder: In which the model learns how to reduce the input dimensions and compress the input data into an encoded representation.

2- Bottleneck: which is the layer that contains the compressed representation of the input data. This is the lowest possible dimensions of the input data.

3- Decoder: In which the model learns how to reconstruct the data from the encoded representation to be as close to the original input as possible.

4- Reconstruction Loss: This is the method that measures measure how well the decoder is performing and how close the output is to the original input.

![alt text](https://github.com/mohan-mj/Autoencoder/blob/master/1.jpg?raw=true)

In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import zipfile

In [0]:
zf = zipfile.ZipFile('ml-1m.zip')
zf.extractall()
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [61]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [62]:
movies.shape

(3883, 3)

In [63]:
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [64]:
users.shape

(6040, 5)

In [65]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [66]:
zf = zipfile.ZipFile('ml-100k.zip')
zf.extractall()
training_set = pd.read_csv('ml-100k/u1.base', delimiter='\t')
training_set.head()

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561


In [0]:
training_set = np.array(training_set, dtype='int')

In [0]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter='\t')
test_set = np.array(test_set, dtype='int')

In [69]:
test_set.shape

(19999, 4)

In [70]:
training_set.shape

(79999, 4)

In [71]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_users

943

In [72]:
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_movies

1682

In [0]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)


In [0]:
# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [0]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

In [0]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


In [81]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(1.7664)
epoch: 2 loss: tensor(1.0965)
epoch: 3 loss: tensor(1.0534)
epoch: 4 loss: tensor(1.0384)
epoch: 5 loss: tensor(1.0311)
epoch: 6 loss: tensor(1.0263)
epoch: 7 loss: tensor(1.0239)
epoch: 8 loss: tensor(1.0220)
epoch: 9 loss: tensor(1.0206)
epoch: 10 loss: tensor(1.0198)
epoch: 11 loss: tensor(1.0189)
epoch: 12 loss: tensor(1.0184)
epoch: 13 loss: tensor(1.0179)
epoch: 14 loss: tensor(1.0177)
epoch: 15 loss: tensor(1.0174)
epoch: 16 loss: tensor(1.0169)
epoch: 17 loss: tensor(1.0166)
epoch: 18 loss: tensor(1.0163)
epoch: 19 loss: tensor(1.0166)
epoch: 20 loss: tensor(1.0160)
epoch: 21 loss: tensor(1.0163)
epoch: 22 loss: tensor(1.0160)
epoch: 23 loss: tensor(1.0156)
epoch: 24 loss: tensor(1.0159)
epoch: 25 loss: tensor(1.0153)
epoch: 26 loss: tensor(1.0158)
epoch: 27 loss: tensor(1.0156)
epoch: 28 loss: tensor(1.0151)
epoch: 29 loss: tensor(1.0130)
epoch: 30 loss: tensor(1.0121)
epoch: 31 loss: tensor(1.0095)
epoch: 32 loss: tensor(1.0093)
epoch: 33 loss: t

In [83]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[0,target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([1682])) that is different to the input size (torch.Size([1, 1682])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


test loss: tensor(0.9549)
